# Basics 7: Grant permissions for Power BI API access

In this lesson we grant our implementation access to the Power BI API.

## Step 1: Define secrets for Power BI API access credentials

Our application needs credentials to gain access to Power BI API. We could just copy-paste them in Jupyter Lab, but for educational purposes we will do this correctly by defining two new secrets named **POWERBI_CLIENT_ID** and **POWERBI_CLIENT_SECRET** in our configuration files.

1. Add secret definitions in **scripts/taito/project.sh** for Taito CLI commands:

```shell
taito_secrets="
  ...
  $taito_project-$taito_env-powerbi.clientId:manual
  $taito_project-$taito_env-powerbi.clientSecret:manual
  ...
"
```

2. Add secret definitions in **docker-compose.yaml** for local development:

```
  data-pipeline-template-worker:
    secrets:
      - POWERBI_CLIENT_ID
      - POWERBI_CLIENT_SECRET
      
  data-pipeline-template-lab:
    secrets:
      - POWERBI_CLIENT_ID
      - POWERBI_CLIENT_SECRET
...

secrets:
  POWERBI_CLIENT_ID:
    file: ./secrets/${taito_env}/${taito_project}-${taito_env}-powerbi.clientId
  POWERBI_CLIENT_SECRET:
    file: ./secrets/${taito_env}/${taito_project}-${taito_env}-powerbi.clientSecret
```

3. Optional: Add secret definitions in `scripts/helm.yaml` for Kubernetes:

```
    worker:
      secrets:
        POWERBI_CLIENT_ID: ${taito_project}-${taito_env}-powerbi.clientId
        POWERBI_CLIENT_SECRET: ${taito_project}-${taito_env}-powerbi.clientSecret
```

TIP: You can find more instructions on secrets and environment variables in [Taito CLI documentation](https://taitounited.github.io/taito-cli/tutorial/06-env-variables-and-secrets/).

## Step 2: Create a shared service principal for accessing Power BI API

Service principal is an user account that is used by one or more applications to access Azure services.

1. Go to https://portal.azure.com.
2. Select **Azure Active Directory**.
3. Select **App registrations**.
4. Select **New registration**.
5. Enter some descriptive name, for example `Power BI Integrations (development)`. Use defaults for the other settings. NOTE: We add **(development)** to the name because this service principal is meant for development and testing purposes only. This way we can grant access to only such datasets that do not contain any confidential data.
6. Register the app by pressing the **Register** button.
7. Go to **Certificates & secrets**.
8. Create a new secret by pressing the **New client secret** button.
9. Enter your project name as the secret name and add environment name as suffix, for example: `my-project-dev`. NOTE: We enter **dev** instead of **local** because we usually use dev environment secrets also in local development.
10. Set secret values to you local environment by running `taito secret rotate powerbi`. Use **Application (client) ID** of overview page as the **clientId**, and the secret value from the **value column** of secret page as **clientSecret**.

## Step 3: Create a new security group for your tenant

Security group is a collection of user accounts, service principals, etc. You need a security group in the next step when you limit access to Power BI API globally on tenant level.

1. Go to https://portal.azure.com.
2. Select Azure Active Directory.
3. Select Groups.
4. Create a new security group for Power BI applications. For example `Power BI Applications (development)`.
5. Add your previously created service principal as group member.

## Step 4: Enable service principal access for Power BI

You need to enable service principal access for Power BI API on tenant level. However, with the previously created security group you can limit the access only for specific service principals.

1. Go to https://app.powerbi.com.
2. Select **Settings -> Admin portal**.
3. Scroll down until you see **Developer settings**.
4. Enable **Allow service principals to use Power BI APIs**.
5. Enter the name of the security group that you previously created.
6. Click **Apply**.

## Step 5: Enable service principal access for Power BI workspace

You need to give permissions for the previously created service principal to access your workspace.

1. Go to https://app.powerbi.com.
2. Select an existing workspace or create a new. It is recommended to create a new as service principal access might not work with old workspaces.
3. Select **Access**.
4. Give the service principal either **Admin** or **Contributor** permissions for the workspace.

TIP: If you cannot see the "Access" option, enable the Premium Per User trial. It will give 60 days trial for free.

## Step 6: Configure Power BI workspace id

1. In Power BI Service, navigate to the workspace you configured previously and copy the group id from the browser address bar.
2. Configure **authority uri** and **group id** in **docker-compose.yaml** for local development. Replace **YOURDOMAIN.COM** with the domain of your Azure tenant.

```
  data-pipeline-template-worker:
    environment:
      POWERBI_AUTHORITY_URI: https://login.microsoftonline.com/YOURDOMAIN.COM
      POWERBI_GROUP_ID: ba40b202-33c2-454d-a54c-ba5d4ca32773
      
  data-pipeline-template-lab:
    environment:
      POWERBI_AUTHORITY_URI: https://login.microsoftonline.com/YOURDOMAIN.COM
      POWERBI_GROUP_ID: ba40b202-33c2-454d-a54c-ba5d4ca32773
```

3. Optional: Configure **authority uri** and **group id** in **scripts/helm.yaml** for Kubernetes:

```
    worker:
      env:
        POWERBI_AUTHORITY_URI: https://login.microsoftonline.com/YOURDOMAIN.COM
        POWERBI_GROUP_ID: ba40b202-33c2-454d-a54c-ba5d4ca32773
```

4. Stop containers with **ctrl-c** and then start them again with `taito start`. This is required for configuration changes to take effect.

TIP: You can find more instructions on environment variables in [Taito CLI documentation](https://taitounited.github.io/taito-cli/tutorial/06-env-variables-and-secrets/).

## Next lesson: [Basics 8: Create a new Power BI push dataset](08.ipynb)